# Video Data Collection from YT API

In [1]:
# Import required modules
import pandas as pd
import requests
import csv
import re
from google.cloud import bigquery

In [ ]:
df = pd.read_csv("data/twitter_yt_videos.csv")
df.head()


,url,author_id,created_at,video_id
0,https://youtu.be/I-3D3pFNet8,852201867031523329,2017-04-24 11:59:59+00:00,I-3D3pFNet8
1,https://www.youtube.com/watch?v=5Vo0rkjLjmI&fe...,92983128,2022-09-25 22:18:58+00:00,5Vo0rkjLjmI
2,https://www.youtube.com/watch?v=56EQ-0b5DSA&fe...,1016734316930486272,2020-11-25 14:50:44+00:00,56EQ-0b5DSA
3,https://www.youtube.com/watch?v=i3aX3JUpxUA&fe...,445724058,2022-03-12 12:48:09+00:00,i3aX3JUpxUA
4,https://www.youtube.com/watch?v=BLx0qPxNrvs&fe...,2901222566,2015-12-04 16:36:26+00:00,BLx0qPxNrvs


In [4]:
df.shape

(789936, 4)

In [5]:
video_ids = list(set(df["video_id"]))
len(video_ids)


560415

In [ ]:
csv_files = [
    "video_data1.csv",
    "video_data2.csv",
    "video_data3.csv",
    "video_data4.csv",
    "video_data5.csv",
]
done_videos = []

for file_name in csv_files:
    df = pd.read_csv(file_name)
    video_ids = df["video_id"].tolist()
    done_videos.extend(video_ids)

with open("completed_ids.txt", "r") as file:
    for line in file:
        done_videos.append(line.strip())

with open("invalid_ids.txt", "r") as file:
    for line in file:
        done_videos.append(line.strip())

done_videos = list(set(done_videos))
len(done_videos)

560410

In [47]:
import os
import csv
import requests


def fetch_video_data(csv_file, api_key, video_ids):
    # Check if the CSV file exists, and if not, create it with the new header
    if not os.path.exists(csv_file):
        with open(csv_file, "w", newline="") as file:
            header = [
                "channel_id",
                "channel_title",
                "playlist_id",
                "video_id",
                "category_id",
                "published_at",
                "title",
                "description",
                "thumbnail",
                "view_count",
                "like_count",
                "favorite_count",
                "comment_count",
                "tags",
            ]
            writer = csv.writer(file)
            writer.writerow(header)

    # existing_videos = set()
    existing_videos = []
    invalid_ids = []

    # Check for existing videos and invalid video IDs
    # with open(csv_file, 'r') as file:
    #     reader = csv.reader(file)
    #     next(reader)  # Skip header row
    #     for row in reader:
    #         existing_videos.add(row[3])  # Video ID is at index 3

    with open("completed_ids.txt", "r") as file:
        for line in file:
            existing_videos.append(line.strip())

    # Open 'invalid_ids.txt' in append mode to avoid overwriting
    with open("invalid_ids.txt", "r") as file:
        for line in file:
            invalid_ids.append(line.strip())

    with open(csv_file, "a", newline="") as file:
        writer = csv.writer(file)
        for video_id in video_ids:
            if video_id in ["nan", "nYkUePQMfkE", "lcOJs8o7kyQ"]:
                continue
            if video_id in invalid_ids:
                continue
            if video_id in existing_videos:
                # print(f"Video {video_id} already fetched. Skipping...")
                continue

            url = f"https://www.googleapis.com/youtube/v3/videos?part=snippet,contentDetails,statistics&key={api_key}&id={video_id}"

            try:
                response = requests.get(url)
                response_data = response.json()

                if "items" in response_data and len(response_data["items"]) > 0:
                    video_info = response_data["items"][0]

                    channel_id = video_info["snippet"]["channelId"]
                    published_at = video_info["snippet"]["publishedAt"]
                    title = video_info["snippet"]["title"]
                    description = video_info["snippet"]["description"]
                    thumbnail = video_info["snippet"]["thumbnails"]["default"]["url"]
                    view_count = video_info["statistics"].get("viewCount", None)
                    like_count = video_info["statistics"].get("likeCount", None)
                    favorite_count = video_info["statistics"].get("favoriteCount", None)
                    comment_count = video_info["statistics"].get("commentCount", None)
                    tags = ",".join(video_info.get("snippet", {}).get("tags", []))
                    channel_title = video_info["snippet"].get("channelTitle", None)
                    category_id = video_info.get("snippet", {}).get("categoryId", None)

                    writer.writerow(
                        [
                            channel_id,
                            channel_title,
                            "",
                            video_id,
                            category_id,
                            published_at,
                            title,
                            description,
                            thumbnail,
                            view_count,
                            like_count,
                            favorite_count,
                            comment_count,
                            tags,
                        ]
                    )
                    existing_videos.append(video_id)
                    with open("completed_ids.txt", "a") as file:
                        file.write(video_id + "\n")
                else:
                    invalid_ids.append(video_id)
                    with open("invalid_ids.txt", "a") as file:
                        file.write(video_id + "\n")
                    print(f"No data found for video {video_id}")

            except Exception as e:
                print(f"An error occurred for video {video_id}: {e}")

    print(f"Video data has been fetched and written to {csv_file} successfully.")

In [33]:
# video_ids = list(filter(lambda x: x not in done_videos, video_ids))
# print(len(video_ids))

In [7]:
from collections import Counter

list1 = video_ids
list2 = done_videos

counter1 = Counter(list1)
counter2 = Counter(list2)

difference_counter = counter1 - counter2

list3 = list(difference_counter.elements())
len(list3)

0

In [ ]:
# Usage
video_ids = list3
csv_file = "data/video_data6.csv"  # CSV file path
api_key = "yt_api_key"  # YouTube API key


In [36]:
fetch_video_data(csv_file, api_key, video_ids)

An error occurred for video nan: unsupported operand type(s) for +: 'float' and 'str'
An error occurred for video -EliglfiNko: need to escape, but no escapechar set
An error occurred for video 4QaZ9KvZdkY: need to escape, but no escapechar set
An error occurred for video xgw6KtsOGfk: need to escape, but no escapechar set
Video data has been fetched and written to video_data6.csv successfully.


In [36]:
# df2 = pd.read_csv("video_data.csv")

In [38]:
# df2.tail()

,channel_id,playlist_id,video_id,published_at,title,description,thumbnail,view_count,like_count,favorite_count,comment_count
113995,UCBAwQnnIY8_BkUEHGiMO34w,aZdREZKt2rs,2013-03-14T20:58:41Z,Side Effects - The Forgotten People (Soundtrac...,Composed by Thomas Newman\nTracklist:\n01. Acu...,https://i.ytimg.com/vi/aZdREZKt2rs/default.jpg,243361,1923.0,0.0,140.0,NaN
113996,UCQW068gtUipAPsOk_grBVFw,nFBGlhS4xBY,2022-01-29T01:52:58Z,Ottawa: Drone view of freedom convoy truck ral...,Stunning drone video of massive convoy of truc...,https://i.ytimg.com/vi/nFBGlhS4xBY/default.jpg,621513,27373.0,0.0,4290.0,NaN
113997,UCJD2JwJNPzi1qcLizxmxbJA,YLbFmdG6uTI,2022-09-12T11:07:15Z,高市・萩生田コンビが滅ぼす日本企業,★９月１９日東京講演会★\n半導体問題が朝鮮戦争までを巻きおこす！？\n日本の未来は？\n米...,https://i.ytimg.com/vi/YLbFmdG6uTI/default.jpg,67416,6391.0,0.0,639.0,NaN
113998,UCg4So70t4SybALDTJhEPHZw,g2l4bdFkvbk,2022-07-28T08:42:37Z,DANGER ROAD - ESIGODINI,Driving from Bulawayo along Esigodini just bef...,https://i.ytimg.com/vi/g2l4bdFkvbk/default.jpg,31,1.0,0.0,0.0,NaN
113999,UCHJAdKXAcZIRvAdoinhApcg,GqNq_TJuOyg,2015-09-17T16:47:57Z,CONJUNTO ALMA GAÚCHA - Ei Tu,Um clássico nativista da virada dos 70 pros 80...,https://i.ytimg.com/vi/GqNq_TJuOyg/default.jpg,72948,1259.0,0.0,82.0,NaN
